In [ ]:
!pip install openpyxl
!pip install selenium

import pandas as pd
import random
import openpyxl
import time 
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By

In [ ]:
input_file_path = "extracted_data_clothing.xlsx"
df = pd.read_excel(input_file_path)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("log-level=3")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--start-miniimized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
driver.get('https://sasb.org/standards/materiality-finder/find/')
search_box = driver.find_element(By.XPATH, '//input[@data-testid="company-search-form-input"]')
search_box.clear()

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.append(['Company Name', 'Sales revenue', 'Found in SASB'])
output_file = 'data_' + str(random.randint(1, 1111)) + '.xlsx'
workbook.save(output_file)

In [ ]:
def append_to_excel(data):
    workbook = openpyxl.load_workbook(output_file)
    sheet = workbook.active
    for row in data:
        sheet.append(row)
    workbook.save(output_file)
        
def remove_right_portion_if_word_found(input_string):
    word = 'Ltd'
    index = input_string.find(word)
    if index != -1:  
        input_string = input_string[:index]  
    return input_string

for ind in df.index:
    company_name = df['Company Name'][ind]
    filtered_name = company_name.replace('.', '').strip()
    filtered_name = remove_right_portion_if_word_found(filtered_name)
    search_box.clear()
    search_box.send_keys(filtered_name.strip())
    driver.find_element(By.XPATH, '//button[@data-testid="company-search-form-submit"]').click()
    time.sleep(3)
    result = driver.find_element(By.XPATH, '//div[@role="status"]').text
    if 'Found' in result:
        append_to_excel([[company_name, df['Sales revenue'][ind], 'Yes']])
        print(f'{company_name} -> Found')
    else:
        append_to_excel([[company_name, df['Sales revenue'][ind], 'No']])
        print(f'{company_name} -> Not Found')